In [6]:
import numpy as np
from sympy.solvers import solve
from sympy import Symbol

In [7]:
x = Symbol('x')
y = Symbol('y')

In [12]:
solve((x**2/10 + y**2/5 + 3*x + 10), y, x)

[{y: -sqrt(-2*x**2 - 60*x - 200)/2}, {y: sqrt(-2*x**2 - 60*x - 200)/2}]

In [11]:
10^2

8